In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import load_iris

iris_data = load_iris()
df = pd.DataFrame(np.c_[ iris_data['data'], iris_data['target'] ],
                    columns=iris_data['feature_names'] + ['target'])

In [31]:
def split_dataset(X, y, test_size, shuffle=True):
    n_samples = X.shape[0]
    permut = np.random.permutation(n_samples) if shuffle else range(n_samples)
    split_point = round((1-test_size)*n_samples)

    X_train, y_train = X[permut[:split_point]], y[permut[:split_point]]
    X_test, y_test = X[permut[split_point:]], y[permut[split_point:]]

    return X_train, X_test, y_train, y_test

In [48]:
def one_hot_encoder(target):
    keys = list(set(target))
    values = list(range(len(keys)))
    class_dict = {key:val for (key, val) in zip(keys, values)}

    target_one_hot = np.zeros((target.shape[0], len(keys)), dtype=int)
    for i, item in enumerate(target):
        target_one_hot[i, class_dict[item]] = 1

    return target_one_hot

In [50]:
X_iris = df.drop(columns=['target']).to_numpy()
y_iris = one_hot_encoder(df['target'].to_numpy())
X_train, X_test, y_train, y_test = split_dataset(X_iris, y_iris, test_size=0.2)